In [1]:
#!pip install lime
#!pip install shap
#!pip install anchor-exp
#!pip install hyperopt

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

import os
import joblib

import warnings
warnings.filterwarnings(action = 'ignore')

C:\ProgramData\Anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\ProgramData\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.NOIJJG62EMASZI6NYURL6JBKM4EVBGM7.gfortran-win_amd64.dll
C:\ProgramData\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
# path to project folder
# please change to your own
PATH = os.getcwd()

dataset = "bike_sharing"
balanced = True
#cls_method = "xgboost" 

random_state = 39

dataset_folder = "%s/%s/" % (PATH, dataset)

In [3]:
#Load and process data
dataset_name = "%s.csv" % (dataset)
dataset_path = dataset_folder + "/datasets/" + dataset_name
data = pd.read_csv( dataset_path )
#data = data.dropna()

# features
class_cols = {"bike_sharing": "total_rental", "breast_cancer": "diagnosis", "compas": "high_risk",
              "diabetes": "Outcome", "facebook": "post_consumers", "housing": "MEDV", "income": "income", "iris": "class", 
              "mushroom": "target", "nursery": "class", "real_estate": "house_price", "solar_flare": "num_flares", 
              "student_scores": "G3", "wine_quality": "quality"}
# if dataset == "diabetes":
#     class_var = "Outcome"
# elif dataset == "breast_cancer":
#     class_var = "diagnosis"

class_var = class_cols[dataset]

# rem_cols = {"bike_sharing": [class_var], "breast_cancer": [class_var, "id"], "compas": [class_var],
#             "diabetes": [class_var], "facebook": [class_var], "housing": [class_var], "income": [class_var, "fnlwgt"], 
#             "iris": [class_var], "mushroom": [class_var], "nursery": [class_var], "real_estate": [class_var], "solar_flare": [class_var], 
#             "student_scores": [class_var], "wine_quality": [class_var]}

#drop_cols = rem_cols[dataset]

# #ID columns
# if dataset == "breast_cancer":
#     drop_cols = [class_var, "id"]
# else:
#     drop_cols = [class_var]
    
feature_names = data.drop(class_var, axis=1).columns.to_list()
#feature_names = feature_names.drop(id_col, axis = 1).columns.to_list()

# balance dataset
if balanced == False:
    classes = data[class_var]
    neg_cases = data[data[class_var] == 0]
    pos_cases = data[data[class_var] == 1]

    if len(neg_cases) > len(pos_cases):
        neg_cases = neg_cases.sample(n=len(pos_cases), random_state = random_state)
    elif len(pos_cases) > len(neg_cases):
        pos_cases = pos_cases.sample(n=len(neg_cases), random_state = random_state)

    balanced_data = [neg_cases, pos_cases]
    balanced_data = pd.concat(balanced_data)

    # check how balanced the classes are
    balanced_data.groupby(class_var).count()
    
else:
    balanced_data = data

In [4]:
#feature_names = balanced_data.drop([class_var], axis=1).columns.tolist()

X = balanced_data[ feature_names ]#.values
Y = balanced_data[class_var]#.values

scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

X = pd.DataFrame(X_scaled, columns = X.columns)
joblib.dump(scaler, dataset_folder+"scaler.joblib")

#generate training and testing data
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=515)
X_validation, X_test, y_validation, y_test = train_test_split(X_test, y_test, test_size=0.5, random_state=515)


X_train.to_csv(dataset_path.replace(".csv", "") + "_Xtrain.csv", sep=";", index = False)
X_test.to_csv(dataset_path.replace(".csv", "") + "_Xtest.csv", sep=";", index = False)
X_validation.to_csv(dataset_path.replace(".csv", "") + "_Xvalidation.csv", sep=";", index = False)
y_train.to_csv(dataset_path.replace(".csv", "") + "_Ytrain.csv", sep=";", index = False)
y_test.to_csv(dataset_path.replace(".csv", "") + "_Ytest.csv", sep=";", index = False)
y_validation.to_csv(dataset_path.replace(".csv", "") + "_Yvalidation.csv", sep=";", index = False)

In [5]:
results_template_test = pd.DataFrame(y_test)
results_template_validation = pd.DataFrame(y_validation)
results_template = pd.concat([results_template_test, results_template_validation])
results_template = results_template.rename(columns = {class_var: "Actual"})

results_template.to_csv(dataset_path.replace(".csv", "") + "_results_template.csv", sep=";", index = False)

In [6]:
X_train

,temp,atemp,hum,windspeed,season_1,season_2,season_3,season_4,yr_0,yr_1,...,weekday_3,weekday_4,weekday_5,weekday_6,workingday_0,workingday_1,weathersit_1,weathersit_2,weathersit_3,weathersit_4
13757,0.714286,0.6818,0.66,0.105325,0.0,0.0,1.0,0.0,0.0,1.0,...,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
14688,0.510204,0.5000,0.77,0.228047,0.0,0.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
3047,0.530612,0.5152,0.60,0.000000,0.0,1.0,0.0,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
7007,0.530612,0.5152,0.64,0.228047,0.0,0.0,0.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
8914,0.326531,0.3182,0.87,0.333373,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15364,0.387755,0.4091,0.71,0.157870,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
17110,0.326531,0.3333,0.66,0.228047,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
6635,0.448980,0.4545,0.88,0.000000,0.0,0.0,0.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
11650,0.653061,0.6212,0.74,0.228047,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0
